<a href="https://colab.research.google.com/github/szymbor2/ATMS597-Project5-GroupG/blob/master/ATMS597_Arka.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

!mkdir '/content/drive/My Drive/project5_data/'

In [0]:
!wget ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-{2000..2019}/64010KSTC{2000..2019}{01..12}.dat
!cp *.dat '/content/drive/My Drive/project5_data/'

In [0]:
import glob
import pandas as pd
import numpy as np
import datetime


for yr in range(2001,2020):
  print('Year = ', yr)
  files = sorted(glob.glob('/content/drive/My Drive/project5_data/64010KSTC'+str(yr)+'*.dat'))

  for ifile in range(0,1):
      print(ifile)
      f = open(files[ifile], 'rb')
      df = pd.DataFrame([str(f.readline()).replace('\\n\'','').replace('b\'','').strip().split(' ')])
      for line in f:
          data = pd.DataFrame([str(f.readline()).replace('\\n\'','').replace('b\'','').strip().split(' ')])
          df = pd.concat([df, data], ignore_index=True)  
      # df = pd.DataFrame([line.strip().split(' ') for line in open(files[ifile], 'r')])
      del df[3]
      n_cols = len(df.columns)
      columns = ['StnNo', 'ContainsDate', 'Time', "Interval", 'Sttn', 'UTC', 'Type', 'Wind', "Weath/Obstr", 'SkyCond', 'Temp',
                    'n_days', 'MSLP', 'RelHum', ' ', 'WndDir/Spd', 'WindMagnetic', 'RMK', 'AO2']
      out_name = files[ifile].replace('/content/drive/My Drive/project5_data/','/content/drive/My Drive/project5_data/reviewed')
      out = open(out_name, 'w')
      out.write('date'+','+'timeofday'+','+'Wind'+','+'Wind_Direction'+','+'Wind_Speed'+','+'Gusts'+','+'Variable_Winds'+','+'Temperature'+','+ 
                        'Dewpoint'+','+'rhum'+','+'hourly'+','+'sixhourly'+','+'dayprecip'+','+'solid'+','+'liquid'+'\n')
      # new_df = pd.DataFrame(columns=new_df_columns)
      cols_already = len(columns)
      cols_new = []
      cols_new = [np.append(cols_new,'X') for i in range(n_cols-cols_already)]
      columns = np.append(columns, cols_new)
      df.columns = columns

      for i in range(0, np.shape(df)[0]): #np.shape(df)[0]
          try:
            day = df['ContainsDate'][i][-8:] 
            time = df['Time'][i]
            date = datetime.datetime.strptime(day+' '+time, '%m/%d/%y %H:%M:%S')
            (h, m, s) = time.split(':')
            timeofday = float(int(h) * 3600 + int(m) * 60 + int(s))/(3600*24)
            # print(date)
            # print(timeofday)
            Gusts = 0
            Wind = 0
            Variable_Winds = 0
            try:
              if "KT" in df['Wind'][i]:
                  Wind = 1
                  if df['Wind'][i][0:3] == 'VRB':
                      Variable_Winds = 1
                  else:
                      Variable_Winds = 0
                      Wind_Direction = int(df['Wind'][i][0:3]) # in tens of degrees from true north
                  Wind_Speed = int(df['Wind'][i][3:5]) # in whole knots (two digits)
                  if df['Wind'][i][5] == 'G':
                      Gusts = 1
                      Gust_Speed = int(df['Wind'][i][6:8]) # speed in whole knots (two digits)
              else:
                  Wind = 0
            except:
              Wind = 0
              Variable_Winds = 0
              Wind_Direction = np.nan
              Gusts=0
              Gust_Speed=0

            Temp = 0
            try:
              if "/" in df['Temp'][i]:
                  Temp = 1
                  Temperature = df['Temp'][i].split('/')[0]
                  if 'M' in Temperature:
                    Temperature = int(Temperature[1:3])
                  else:
                    Temperature = int(Temperature)
                  Dewpoint = df['Temp'][i].split('/')[1]
                  if 'M' in Dewpoint:
                    Dewpoint = int(Dewpoint[1:3])
                  else:
                    Dewpoint = int(Dewpoint) 
              else:
                  Temp = 0  
              try:
                rhum = int(df['RelHum'][i])
              except:
                rhum = np.nan
              if (rhum>100):
                rhum = np.nan
            except:
              Temp=0
              Temperature=np.nan
              Dewpoint=np.nan
              rhum=np.nan

            for n in range(cols_already, n_cols): #n_cols
              # for j in range(len(df.iloc[:,n])):
                element = df.iloc[i,n]
                try:
                  if element[0]=='P' or element[0]=='6' or element[0]=='7':
                    if element[1]=='0':
                      if element[0]=='P':
                        hourly = float(element[1:])/100.
                      elif element[0]=='6':
                        sixhourly = float(element[1:])/100.
                      elif element[0]=='7':
                        dayprecip = float(element[1:])/100.
                  else:
                    hourly = np.nan
                    sixhourly = np.nan
                    dayprecip = np.nan
                except:
                    hourly = np.nan
                    sixhourly = np.nan
                    dayprecip = np.nan

            frozen_flags = ['SN', 'SG', 'IC', 'PL', 'GR', 'GS']
            liquid_flags = ['DZ', 'RA']
            try:
                if np.logical_or(any(flags in df['Weath/Obstr'][i] for flags in frozen_flags), any(flags in df['Temp'][i] for flags in frozen_flags)):
                  solid = 1
                elif np.logical_or(any(flags in df['Weath/Obstr'][i] for flags in liquid_flags), any(flags in df['Temp'][i] for flags in liquid_flags)):
                  liquid = 1
                else:
                  solid = 0
                  liquid = 0
            except:
                  solid = np.nan
                  liquid = np.nan

            # print('Wind, Wind_Direction, Wind_Speed, Gusts, Variable_Winds, Temp, Temperature, Dewpoint, rhum, hourly, sixhourly, daily:') 
            # print(str(date)+','+str(timeofday)+','+str(Wind)+','+str(Wind_Direction)+','+str(Wind_Speed)+','+str(Gusts)+','+str(Variable_Winds)
            # +','+str(Temperature)+','+str(Dewpoint)+','+str(rhum)+','+str(hourly)+','+str(sixhourly)+','+str(dayprecip)+','+str(solid)+','+str(liquid)+'\n')
            out.write(str(date)+','+str(timeofday)+','+str(Wind)+','+str(Wind_Direction)+','+str(Wind_Speed)+','+str(Gusts)+','+str(Variable_Winds)
            +','+str(Temperature)+','+str(Dewpoint)+','+str(rhum)+','+str(hourly)+','+str(sixhourly)+','+str(dayprecip)+','+str(solid)+','+str(liquid)+'\n')
          except:
            continue
      out.close()

<<-----Placeholder stuff below. Actual code above --------------------->>

In [0]:
infile = pd.read_csv(out_name, header=0)
infile

In [0]:
f = open(files[ifile], 'rb')
df = pd.DataFrame([str(f.readline()).replace('\\n\'','').replace('b\'','').strip().split(' ')])
for line in f:
        # print(line.decode(errors='ignore'))
        data = pd.DataFrame([str(f.readline()).replace('\\n\'','').replace('b\'','').strip().split(' ')])
        df = pd.concat([df, data])
df

In [0]:
f = open(files[ifile], 'rb')
ph = str(f.readline()).replace('\\n\'','').replace('b\'','').strip().split(' ')
# ph[len(ph)-1].replace('\\n','')
ph